# Чат-клієнт з використанням Tkinter та сокетів

## Мета
Створити настільний додаток на Python за допомогою Tkinter та сокетів для реалізації функціоналу чат-клієнта та сервера.

## Основні вимоги

### Інтерфейс користувача
- Створіть вікно додатку з використанням Tkinter для клієнта.
- Реалізуйте текстове поле для введення повідомлень і кнопку надсилання.
- Відображайте вхідні та вихідні повідомлення в області перегляду чату.
- Інтерфейс має бути адаптивним *(адекватно виглядати на різних розмірах вікна)*

### Мережева взаємодія
- Реалізуйте базовий сервер на сокетах, який може приймати повідомлення від клієнтів і пересилати їх усім підключеним клієнтам.
- Клієнт повинен підключатися до сервера і надсилати/отримувати повідомлення через сокети.

### Додаткове завдання
- Реалізуйте вибір імені користувача та відображайте його в чаті.
- Додайте логування усіх повідомлень у текстовий файл як на боці клієнта, так і сервера.
- Забезпечте можливість підключення декількох клієнтів до сервера одночасно.

## Критерії оцінки
- Стабільність з'єднання та коректність роботи мережевого взаємодії.
- Якість та зручність користувацького інтерфейсу.
- Реалізація багатокористувацького підключення та додаткових функцій.

## Приклад коду

### Приклад серверу:

```python
import socket
import threading

def client_thread(conn, addr):
    print(f"Connected by {addr}")
    while True:
        message = conn.recv(1024).decode()
        if not message:
            break
        print(f"Received from {addr}: {message}")
    conn.close()

def start_server():
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind(('localhost', 12345))
    server.listen()

    while True:
        conn, addr = server.accept()
        thread = threading.Thread(target=client_thread, args=(conn, addr))
        thread.start()

if __name__ == "__main__":
    start_server()
```

### Приклад клієнта:

```python
import socket

def send_message():
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.connect(('localhost', 12345))

    while True:
        message = input("Your message: ")
        if message.lower() == 'quit':
            break
        client.send(message.encode())

    client.close()

if __name__ == "__main__":
    send_message()
```


In [3]:
import tkinter as tk
from tkinter import scrolledtext
import socket
import threading

window = tk.Tk()
window.title("Chat")
window.geometry("400x500")

def send_message():
    message = entry.get()
    if message:
        full_message = f"{name}: {message}"
        client.send(full_message.encode())
        chat_display.insert(tk.END, full_message + "\n")
        entry.delete(0, tk.END)
    else:
       full_message = ""
       client.send(full_message.encode()) 

def receive_message():
    while True:
        try:
            message = client.recv(1024).decode()
            chat_display.insert(tk.END, message + "\n")
        except ConnectionResetError:
            break
        
name = None

def get_name():
    global name
    name = choose_name.get()
    secondary_window.destroy()
    send_message()


secondary_window = tk.Toplevel(window)
secondary_window.title("Choose name")
secondary_window.geometry('250x70')

choose_name = tk.Entry(secondary_window, width=15)
choose_name.grid(row=2, column=4, padx=20, pady=20, sticky="ew")

confirm_button = tk.Button(secondary_window, text='Confirm', width=8, command=get_name)
confirm_button.grid(row=2, column=6, padx=12, pady=12, sticky="ew")

chat_display = scrolledtext.ScrolledText(window, wrap=tk.WORD, width=40, height=10)
chat_display.grid(row=0, column=0, columnspan=2, padx=10, pady=10, sticky="nsew")

entry = tk.Entry(window, width=30)
entry.grid(row=1, column=0, padx=10, pady=10, sticky="ew")

def send_button_press(event):
    send_button.invoke()

send_button = tk.Button(window, text="Send", command=send_message, width=10)
send_button.grid(row=1, column=1, padx=10, pady=10, sticky="e")

entry.bind('<Return>', send_button_press)

window.columnconfigure(0, weight=1)
window.rowconfigure(0, weight=1)


client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect(('localhost', 12345))
server_thread = threading.Thread(target=receive_message)
server_thread.start()
window.mainloop()